# Objective of This Part: Dataset Merging for Enriched Fraud Detection
In this part of the exercise, the goal is to merge two datasets to create a more enriched and informative dataset for fraud detection. By combining transactional data (e.g., from fraudTrain.csv) with additional information (such as user profiles, merchant details, or geographic risk indicators), we aim to provide the model with more context that may help it better distinguish between fraudulent and legitimate transactions.

This step involves:

* Identifying common keys between datasets (e.g., user, cc_num, or merchant),

* Ensuring structural compatibility (matching granularity, time periods, and data formats),

* Performing a safe and meaningful join,

* Validating the completeness and integrity of the merged result.

Merging datasets properly can reveal hidden patterns, enhance feature richness, and ultimately boost model performance in identifying fraud.

# Logigramme – Fusion de deux datasets

Début  
  │  
  ▼  
1. Identifier la clé de jointure  
(ex : `user`, `cc_num`, `merchant`, etc.)  
  │  
  ▼  
2. Vérifier la qualité de la clé  
- Doublons ?  
- Types compatibles ?  
- Clés correspondantes ?  
  │  
  ▼  
3. Fusionner les datasets  
`pd.merge(df1, df2, how='left', on='clé')`  
  │  
  ▼  
4. Nettoyer le résultat  
- Gérer les `NaN`  
- Supprimer doublons  
- Vérifier dimensions  
  │  
  ▼  
5. Analyse exploratoire des nouvelles colonnes  
- Statistiques  
- Visualisations  
- Corrélations  
  │  
  ▼  
Fin – Dataset enrichi prêt pour la modélisation


## Première étape d’un merge entre deux datasets :

a. Inspecter les colonnes de chaque dataset :

In [4]:
import pandas as pd

# Chargement des fichiers CSV
df1 = pd.read_csv("creditcard_dataset1.csv")
df2 = pd.read_csv("fraudTrain_dataset2.csv")

In [5]:
# Affichage des colonnes
print("df1 columns:", df1.columns.tolist())
print("df2 columns:", df2.columns.tolist())

df1 columns: ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class']
df2 columns: ['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud']


 b. Chercher une ou plusieurs colonnes communes/logiques :

Objectif de cette étape :
➡️ Identifier une clé de correspondance fiable entre les deux datasets.
Sans ça, aucune jointure n’a de sens.

| `df1`                            | `df2`       | 💬 Remarques                                       |
| -------------------------------- | ----------- | -------------------------------------------------- |
| `Time`                           | `unix_time` | même logique temporelle (à confirmer avec un test) |
| `Amount`                         | `amt`       | même donnée, noms différents                       |
| `Class`                          | `is_fraud`  | même signification (étiquette binaire de fraude)   |
| *(aucune autre colonne directe)* | `cc_num`    | **clé candidate** pour le `merge`                  |


C’est la seule vraie colonne d’identifiant présente dans les deux datasets.

 Vérification recommandée avant merge :

In [6]:
df1['cc_num'] = df2['cc_num']  # seulement si tu confirmes qu’ils correspondent 1:1
print(df1['cc_num'].duplicated().sum())  # doit être 0
print(df2['cc_num'].duplicated().sum())


283871
1295692


 Option 1 : Clé composite cc_num + Time
df1 contient cc_num et Time (ou unix_time)

df2 contient cc_num et unix_time

Teste cette correspondance :

In [7]:
df1['unix_time'] = df1['Time'].astype(int)  # si Time est exprimé en secondes
merged = pd.merge(df1, df2, how='inner', on=['cc_num', 'unix_time'])
print(merged.shape)


(0, 54)


Il n’y a aucune combinaison exacte (cc_num, unix_time) commune aux deux datasets.

Solution réaliste : ajuster les données pour créer une clé de jointure approximative.

Option efficace : arrondir Time dans df1 pour se rapprocher de unix_time dans df2

In [8]:
df1['approx_unix_time'] = df1['Time'].astype(int) + df2['unix_time'].min()


In [9]:
df1['key'] = df1['cc_num'].astype(str) + "_" + df1['approx_unix_time'].astype(str)
df2['key'] = df2['cc_num'].astype(str) + "_" + df2['unix_time'].astype(str)

df_merged = pd.merge(df1, df2, how='inner', on='key')
print(df_merged.shape)


(10, 58)


Parfait, (10, 58) signifie que la clé composite cc_num + unix_time a permis de trouver 10 correspondances exactes entre df1 et df2.

Conclusion :
* approche fonctionne
* nous avons maintenant un dataset enrichi de 10 lignes prêtes pour exploration ou modélisation.

1. Vérifier la qualité des données fusionnées :

In [10]:
df_merged.info()
df_merged.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 58 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Time                   10 non-null     float64
 1   V1                     10 non-null     float64
 2   V2                     10 non-null     float64
 3   V3                     10 non-null     float64
 4   V4                     10 non-null     float64
 5   V5                     10 non-null     float64
 6   V6                     10 non-null     float64
 7   V7                     10 non-null     float64
 8   V8                     10 non-null     float64
 9   V9                     10 non-null     float64
 10  V10                    10 non-null     float64
 11  V11                    10 non-null     float64
 12  V12                    10 non-null     float64
 13  V13                    10 non-null     float64
 14  V14                    10 non-null     float64
 15  V15      

Time                     0
V1                       0
V2                       0
V3                       0
V4                       0
V5                       0
V6                       0
V7                       0
V8                       0
V9                       0
V10                      0
V11                      0
V12                      0
V13                      0
V14                      0
V15                      0
V16                      0
V17                      0
V18                      0
V19                      0
V20                      0
V21                      0
V22                      0
V23                      0
V24                      0
V25                      0
V26                      0
V27                      0
V28                      0
Amount                   0
Class                    0
cc_num_x                 0
unix_time_x              0
approx_unix_time         0
key                      0
Unnamed: 0               0
trans_date_trans_time    0
c

Parfait, tu disposes maintenant d’un dataset enrichi avec :

✅ les variables anonymisées (V1–V28), Amount, Class

✅ les métadonnées réelles : merchant, category, cc_num, dob, job, is_fraud, etc.

1. Nettoyage des colonnes redondantes
Tu as des doublons :

cc_num_x / cc_num_y

amt / Amount

unix_time_x, unix_time_y, approx_unix_time

Class / is_fraud

🔧 Garde uniquement les colonnes utiles et unifie :

In [11]:
df_cleaned = df_merged.copy()

df_cleaned = df_cleaned.drop(columns=[
    'cc_num_y', 'amt', 'unix_time_x', 'unix_time_y',
    'approx_unix_time', 'Amount', 'Class'  # on garde 'is_fraud'
])


2. Normalisation des types et formatage des dates :

In [12]:
df_cleaned['trans_date_trans_time'] = pd.to_datetime(df_cleaned['trans_date_trans_time'])
df_cleaned['dob'] = pd.to_datetime(df_cleaned['dob'])


3. Création de variables utiles :

In [13]:
df_cleaned['age'] = df_cleaned['trans_date_trans_time'].dt.year - df_cleaned['dob'].dt.year
df_cleaned['hour'] = df_cleaned['trans_date_trans_time'].dt.hour


Etape : Encodage des variables catégorielles pour modélisation
Tu as des colonnes object (texte) comme :

merchant, category, gender, job, state, city, etc.

Étape : Encodage one-hot simplifié ou label encoding

In [14]:
from sklearn.preprocessing import LabelEncoder

df_encoded = df_cleaned.copy()
for col in ['gender', 'category', 'state']:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])


**Raisons claires et argumentées** pour continuer l'exercice avec le **`creditcard_dataset1`** (le dataset original issu de Kaggle) :

---

### ✅ 1. **Volume suffisant pour apprentissage**

* `creditcard_dataset1` contient **284 807 transactions**, dont **492 fraudes** (soit \~0.17%).
* C’est suffisant pour :

  * entraîner des modèles robustes,
  * effectuer un **split train/test fiable**,
  * et appliquer des techniques d’équilibrage (ex. SMOTE).

---

### ✅ 2. **Nettoyé et prêt à l’emploi**

* Les colonnes sont **numériques** (V1–V28, Amount, Time, Class).
* Aucune donnée textuelle ni bruit inutile.
* Idéal pour PCA, RandomForest, XGBoost ou réseaux neuronaux.

---

### ✅ 3. **Aligné avec l'objectif du hackathon**

* Le sujet impose : **fraude sur transactions de carte bancaire**.
* Ce dataset simule exactement cela, avec anonymisation réaliste (PCA sur features).

---

### ✅ 4. **Benchmark connu**

* Il est utilisé comme **référence dans la recherche et les publications**.
* Tu pourras **comparer tes scores AUC, recall, etc.** avec d’autres solutions publiées.

---

### ✅ 5. **Adapté à un cycle complet de modélisation**

* Tu peux y appliquer :

  * **EDA**
  * **Modélisation supervisée**
  * **Techniques d’échantillonnage**
  * **Évaluation stricte sur données déséquilibrées**
  * Et même créer un **dashboard de suivi de fraude (PowerBI/Tableau)**

---

### 📌 Conclusion :

Le dataset enrichi était utile pour tester le **merge et l’enrichissement contextuel**,
mais le `creditcard_dataset1` est **indispensable pour la modélisation sérieuse**.

Souhaites-tu qu’on reprenne directement un pipeline complet avec ce dataset ?
